In [25]:
import pandas as pd
from sqlalchemy import create_engine, text
import pymysql
pymysql.install_as_MySQLdb()
import numpy as np

In [2]:
db_type = 'mysql'
host = 'host'
port = 'port'
db_name = 'Pizza_DB'
username = 'username'
password = 'password'

engine = create_engine(f'{db_type}+pymysql://{username}:{password}@{host}:{port}/{db_name}')

conn = engine.connect()

In [3]:
file = "pizza_sales.csv"
df = pd.read_csv(file)

df.to_sql(name='pizza_sales', con=engine, if_exists='replace', index=False)

48620

In [4]:
queries = text("""
    ALTER TABLE pizza_sales 
    MODIFY pizza_id INT,
    MODIFY order_id INT,
    MODIFY pizza_name_id VARCHAR(50),
    MODIFY order_date VARCHAR(50),
    MODIFY order_time VARCHAR(50),
    MODIFY pizza_size VARCHAR(50),
    MODIFY pizza_category VARCHAR(50),
    MODIFY pizza_ingredients VARCHAR(200),
    MODIFY pizza_name VARCHAR(50)
    """)

conn.execute(queries)

In [5]:
pizza_sales = pd.read_sql("SELECT * FROM pizza_sales", conn)

pizza_sales.head()

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1,1,hawaiian_m,1,01-01-2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2,2,classic_dlx_m,1,01-01-2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3,2,five_cheese_l,1,01-01-2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4,2,ital_supr_l,1,01-01-2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5,2,mexicana_m,1,01-01-2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [6]:
pizza_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pizza_id           48620 non-null  int64  
 1   order_id           48620 non-null  int64  
 2   pizza_name_id      48620 non-null  object 
 3   quantity           48620 non-null  int64  
 4   order_date         48620 non-null  object 
 5   order_time         48620 non-null  object 
 6   unit_price         48620 non-null  float64
 7   total_price        48620 non-null  float64
 8   pizza_size         48620 non-null  object 
 9   pizza_category     48620 non-null  object 
 10  pizza_ingredients  48620 non-null  object 
 11  pizza_name         48620 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 4.5+ MB


In [7]:
pizza_sales.describe()

,pizza_id,order_id,quantity,unit_price,total_price
count,48620.000000,48620.000000,48620.000000,48620.000000,48620.000000
mean,24310.500000,10701.479761,1.019622,16.494132,16.821474
std,14035.529381,6180.119770,0.143077,3.621789,4.437398
min,1.000000,1.000000,1.000000,9.750000,9.750000
25%,12155.750000,5337.000000,1.000000,12.750000,12.750000
50%,24310.500000,10682.500000,1.000000,16.500000,16.500000
75%,36465.250000,16100.000000,1.000000,20.250000,20.500000
max,48620.000000,21350.000000,4.000000,35.950000,83.000000


---

<b>Total Revenue</b>

In [8]:
total_revenue = pd.read_sql("SELECT SUM(total_price) AS Total_Revenue FROM pizza_sales", conn)
    
total_revenue

,Total_Revenue
0,817860.05


---


<b>Average Order Value</b>

In [9]:
avg_order_value = pd.read_sql("SELECT SUM(total_price) / COUNT(DISTINCT order_id) AS Avg_Order_Value FROM pizza_sales", conn)

avg_order_value   

,Avg_Order_Value
0,38.307262


---

<b>Total Pizzas Sold</b>

In [10]:
Total_Pizzas_Sold = pd.read_sql("SELECT SUM(quantity) AS Total_Pizzas_Sold FROM pizza_sales", conn)

Total_Pizzas_Sold

,Total_Pizzas_Sold
0,49574.0


---

<b>Total Orders</b>

In [11]:
Total_Orders = pd.read_sql("SELECT COUNT(DISTINCT order_id) AS Total_Orders FROM pizza_sales", conn)

Total_Orders

,Total_Orders
0,21350


---

<b>Average Pizzas Per Order</b>

In [12]:
Avg_Pizzas_Per_Order = pd.read_sql("SELECT CAST(SUM(quantity) AS DECIMAL(10,2)) / \
                                        CAST(COUNT(DISTINCT order_id) AS DECIMAL(10,2)) \
                                        AS Avg_Pizzas_Per_Order \
                                    FROM pizza_sales", conn)
# Avg_Pizzas_Per_Order = pd.read_sql("SELECT CAST(CAST(SUM(quantity) AS DECIMAL(10,2)) / CAST(COUNT(DISTINCT order_id) AS DECIMAL(10,2)) as DECIMAL(10,2)) AS Avg_Pizzas_Per_Order FROM pizza_sales", conn)

Avg_Pizzas_Per_Order

,Avg_Pizzas_Per_Order
0,2.321967


---

<b>Hourly Trend for Total Pizzas Sold</b>

In [13]:
Hourly_Trend = pd.read_sql("SELECT EXTRACT(HOUR FROM order_time) AS order_hours, \
                                SUM(quantity) AS Total_Pizzas_Sold, \
                                pizza_category AS Pizza_Category \
                            FROM pizza_sales \
                            GROUP BY EXTRACT(HOUR FROM order_time), pizza_category \
                            ORDER BY EXTRACT(HOUR FROM order_time)", conn)

Hourly_Trend.head()

,order_hours,Total_Pizzas_Sold,Pizza_Category
0,9,2.0,Classic
1,9,1.0,Supreme
2,9,1.0,Veggie
3,10,7.0,Chicken
4,10,6.0,Classic


In [14]:
pizza_sales['pizza_category']

0        Classic
1        Classic
2         Veggie
3        Supreme
4         Veggie
          ...   
48615    Chicken
48616     Veggie
48617    Classic
48618     Veggie
48619    Chicken
Name: pizza_category, Length: 48620, dtype: object

---

<b>Weekly Trend for Total Orders</b>

In [15]:
query = text("SELECT WEEK(STR_TO_DATE(order_date, '%d-%m-%Y'), 1) AS week_number, \
                     YEAR(STR_TO_DATE(order_date, '%d-%m-%Y')) AS Order_Year, \
                     COUNT(DISTINCT order_id) AS Total_Orders \
              FROM pizza_sales \
              GROUP BY Order_Year, week_number \
              ORDER BY Order_Year, week_number")

Weekly_Trend = pd.read_sql(query, conn)

Weekly_Trend.head()

,week_number,Order_Year,Total_Orders
0,1,2015,254
1,2,2015,427
2,3,2015,400
3,4,2015,415
4,5,2015,436


---

<b>Percentage of Sales by Pizza Category</b>

In [16]:
query = text("SELECT pizza_category, \
                SUM(total_price) as Total_Sales, \
                (SUM(total_price) * 100 )/ \
                (SELECT SUM(total_price) FROM pizza_sales WHERE MONTH(STR_TO_DATE(order_date, '%d-%m-%Y')) = 1) AS PCT \
            FROM pizza_sales \
            WHERE MONTH(STR_TO_DATE(order_date, '%d-%m-%Y')) = 1 \
            GROUP BY pizza_category \
            ORDER BY PCT DESC")

PCT = pd.read_sql(query, conn)

PCT

,pizza_category,Total_Sales,PCT
0,Classic,18619.40,26.677919
1,Supreme,17929.75,25.689787
2,Veggie,17055.40,24.437016
3,Chicken,16188.75,23.195278


---

<b>Percentage of Sales by Pizza Size</b>

In [17]:
query = text("SELECT pizza_size, \
                CAST(SUM(total_price) AS DECIMAL(10,2)) as Total_Sales, \
                CAST((SUM(total_price) * 100 )/ \
                (SELECT SUM(total_price) FROM pizza_sales \
                    WHERE QUARTER(STR_TO_DATE(order_date, '%d-%m-%Y')) = 1) AS DECIMAL(10,2)) AS PCT_Size \
            FROM pizza_sales \
            WHERE QUARTER(STR_TO_DATE(order_date, '%d-%m-%Y')) = 1 \
            GROUP BY pizza_size \
            ORDER BY PCT_Size DESC")

PCT_Size_sql = pd.read_sql(query, conn)

PCT_Size_sql

,pizza_size,Total_Sales,PCT_Size
0,L,95229.65,46.37
1,M,61159.00,29.78
2,S,45384.25,22.10
3,XL,3289.50,1.60
4,XXL,287.60,0.14


---

<b>Top 5 Best Sellers by Revenue</b>

In [18]:
query = text("SELECT pizza_name, \
                CAST(SUM(total_price) AS DECIMAL(10,2)) AS Total_Revenue \
            FROM pizza_sales \
            GROUP BY pizza_name \
            ORDER BY Total_Revenue DESC \
            LIMIT 5")

Best_Sellers_Revenue = pd.read_sql(query, conn)

Best_Sellers_Revenue

,pizza_name,Total_Revenue
0,The Thai Chicken Pizza,43434.25
1,The Barbecue Chicken Pizza,42768.00
2,The California Chicken Pizza,41409.50
3,The Classic Deluxe Pizza,38180.50
4,The Spicy Italian Pizza,34831.25


---

<b>Top 5 Best Sellers by Total Quantity</b>

In [19]:
query = text("SELECT pizza_name, \
                SUM(quantity) AS Total_Quantity \
            FROM pizza_sales \
            GROUP BY pizza_name \
            ORDER BY Total_Quantity DESC \
            LIMIT 5")

Best_Sellers_Quantity = pd.read_sql(query, conn)

Best_Sellers_Quantity

,pizza_name,Total_Quantity
0,The Classic Deluxe Pizza,2453.0
1,The Barbecue Chicken Pizza,2432.0
2,The Hawaiian Pizza,2422.0
3,The Pepperoni Pizza,2418.0
4,The Thai Chicken Pizza,2371.0


---

<b>Top 5 Best Sellers by Total Orders</b>

In [20]:
query = text("SELECT pizza_name, \
                COUNT(DISTINCT order_id) AS Total_Orders \
            FROM pizza_sales \
            GROUP BY pizza_name \
            ORDER BY Total_Orders DESC \
            LIMIT 5")

Best_Sellers_Orders = pd.read_sql(query, conn)

Best_Sellers_Orders

,pizza_name,Total_Orders
0,The Classic Deluxe Pizza,2329
1,The Hawaiian Pizza,2280
2,The Pepperoni Pizza,2278
3,The Barbecue Chicken Pizza,2273
4,The Thai Chicken Pizza,2225


---

<b>Bottom 5 Worst Sellers by Revenue</b>

In [21]:
query = text("SELECT pizza_name, \
                CAST(SUM(total_price) AS DECIMAL(10,2)) AS Total_Revenue, \
                SUM(quantity) AS Total_Quantity, \
                COUNT(DISTINCT order_id) AS Total_Orders \
            FROM pizza_sales \
            GROUP BY pizza_name \
            ORDER BY Total_Revenue ASC \
            LIMIT 5")

Worst_Sellers_Revenue = pd.read_sql(query, conn)

Worst_Sellers_Revenue

,pizza_name,Total_Revenue,Total_Quantity,Total_Orders
0,The Brie Carre Pizza,11588.50,490.0,480
1,The Green Garden Pizza,13955.75,997.0,976
2,The Spinach Supreme Pizza,15277.75,950.0,918
3,The Mediterranean Pizza,15360.50,934.0,912
4,The Spinach Pesto Pizza,15596.00,970.0,945


---

<b>Bottom 5 Worst Sellers by Total Quantity</b>

In [22]:
query = text("SELECT pizza_name, \
                SUM(quantity) AS Total_Quantity \
            FROM pizza_sales \
            GROUP BY pizza_name \
            ORDER BY Total_Quantity ASC \
            LIMIT 5")

Worst_Sellers_Quantity = pd.read_sql(query, conn)

Worst_Sellers_Quantity

,pizza_name,Total_Quantity
0,The Brie Carre Pizza,490.0
1,The Mediterranean Pizza,934.0
2,The Calabrese Pizza,937.0
3,The Spinach Supreme Pizza,950.0
4,The Soppressata Pizza,961.0


---

<b>Bottom 5 Worst Sellers by Total Orders</b>

In [23]:
query = text("SELECT pizza_name, \
                COUNT(DISTINCT order_id) AS Total_Orders \
            FROM pizza_sales \
            GROUP BY pizza_name \
            ORDER BY Total_Orders ASC \
            LIMIT 5")

Worst_Sellers_Orders = pd.read_sql(query, conn)

Worst_Sellers_Orders

,pizza_name,Total_Orders
0,The Brie Carre Pizza,480
1,The Mediterranean Pizza,912
2,The Spinach Supreme Pizza,918
3,The Calabrese Pizza,918
4,The Chicken Pesto Pizza,938


---

In [24]:
conn.close()